In [22]:
import requests
import json
import pickle



def get_bne_wikidata(max_number=100):
    key = "bne"
    value = "wikidata"
    endpoint = """https://query.wikidata.org/bigdata/namespace/wdq/sparql?query="""
    
    query = """PREFIX wikibase: <http://wikiba.se/ontology>
    SELECT ?bne ?wikidata
    WHERE { 
        ?wikidata wdt:P950 ?bne .
    }
    LIMIT """+str(max_number)
    
    headers = {"Accept" : "application/json"}
    response = requests.get(endpoint+query, headers=headers)
    json_data = response.json()['results']['bindings']
    return [(item[key]['value'], item[value]['value']) for item in json_data]


datos = get_bne_wikidata()
for key,value in datos:
    print(key,"\t",value)

f = open("wikidatamap.bin", "wb+")
pickle.dump(datos, f)

XX1149955 	 http://www.wikidata.org/entity/Q42
XX1157500 	 http://www.wikidata.org/entity/Q23
XX1601283 	 http://www.wikidata.org/entity/Q207
XX450924 	 http://www.wikidata.org/entity/Q183
XX865983 	 http://www.wikidata.org/entity/Q297
XX1022209 	 http://www.wikidata.org/entity/Q368
XX1026401 	 http://www.wikidata.org/entity/Q1405
XX1044878 	 http://www.wikidata.org/entity/Q3374
XX1085266 	 http://www.wikidata.org/entity/Q1124
XX1108831 	 http://www.wikidata.org/entity/Q1149
XX1146791 	 http://www.wikidata.org/entity/Q747
XX1155787 	 http://www.wikidata.org/entity/Q859
XX1156883 	 http://www.wikidata.org/entity/Q1868
XX1190528 	 http://www.wikidata.org/entity/Q1056
XX1195983 	 http://www.wikidata.org/entity/Q1488
XX1202674 	 http://www.wikidata.org/entity/Q1267
XX1220179 	 http://www.wikidata.org/entity/Q1317
XX1253209 	 http://www.wikidata.org/entity/Q2667
XX1266593 	 http://www.wikidata.org/entity/Q873
XX1302720 	 http://www.wikidata.org/entity/Q2573
XX1475818 	 http://www.wikidata.o

['http:', '', 'www.wikidata.org', 'entity', 'Qxxx']

In [32]:
import importlib
import pickle
import dataset
importlib.reload(dataset)

   
datos = dataset.Dataset()
query2 = """PREFIX wikibase: <http://wikiba.se/ontology>
construct { ?wikidata ?predicate ?object . ?object ?predicate2 ?object2 . ?object2 ?predicate3 ?object3 }
WHERE { ?wikidata wdt:P950 ?bne .
?wikidata ?predicate ?object .
?object ?predicate2 ?object2 .
?object2 ?predicate3 ?object3
} LIMIT 1000
"""
datos.load_dataset_from_query(query2, only_uri=False)
datos.show(verbose=True)

datos.save_to_binary("aquimismo2")

#print(len(jsonlist))
#jsonlist

153 entities, 84 relations, 153 tripletas

Entities (153):
http://www.wikidata.org/entity/statement/q23-02AA80F6-E20C-4F66-B044-1501C8CA4CB7
http://www.wikidata.org/entity/Q23
http://www.wikidata.org/entity/Q30
http://www.wikidata.org/entity/statement/q23-935f9100-47ca-f387-7946-45f9db09e81f
http://www.wikidata.org/entity/Q5
http://www.wikidata.org/entity/statement/Q5-24bf2e5a-46ab-68de-c159-9c89a3c896d1
http://www.wikidata.org/entity/statement/Q5-d3a4fa58-4f14-cef5-c1e7-2626fc6dac25
http://www.wikidata.org/entity/statement/q5-3ff6552a-41f7-0650-dbf3-8b6be2d8dc9a
http://www.wikidata.org/entity/statement/Q5-90121333-4598-0dde-c207-4cd2a39d5098
http://www.wikidata.org/entity/statement/Q5-31A51250-068A-435A-AC90-164EECAC9C03
http://www.wikidata.org/entity/statement/q5-AC1AD522-4529-4348-8A9A-5FF22A14A903
http://www.wikidata.org/entity/statement/Q5-1bbe2872-4dd6-0a71-7faf-c3ceff34bb4e
http://www.wikidata.org/entity/statement/Q5-4D513F46-E17D-4DCA-995F-2FEDD058F841
http://www.wikidata.org/e

True